In [ ]:
import math
import requests
import folium
from geopy.geocoders import Nominatim


def calcular_distancia(lat1, lon1, lat2, lon2):
    R = 6371  # Raio da Terra em km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# função que calcula a distância de um ponto a outro através da (fórmula de Haversine)

def geolocalizar(latitude, longitude):
    url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}'
    headers = {"User-Agent": "AlertaDesastresApp/1.0 (seuemail@gmail.com.br)" }

    # 'User-Agent' exigência da API

# Traz os dados da localização do usuário (rua, cidade, CEP)

    try:
        resposta = requests.get(url, headers=headers)
        resposta.raise_for_status()
        data = resposta.json()
        return data.get("display_name", "Endereço desconhecido")
    except Exception as erro:
        print(f"Erro ao buscar endereço: {erro}")
        return "Endereço não identificado"
    
    # Exibe endereço desconhecido junto da anormalidade em casos de mata, floresta ou situações que não possuem endereço certo

    # API necessita de internet, caso não haja acesso, ele exibe o erro

    # Transforma os dados da API em json para leitura de dados para exibição no código e retorna o valor display_name (vindo do banco de dados)
    
def obter_coordenadas(endereco):
    geolocator = Nominatim(user_agent="AlertaDesastresApp")
    location = geolocator.geocode(endereco)
    if location:
        return location.latitude, location.longitude
    else:
        return None, None
    
    # Obtem as coordenadas do local através do endereço inserido pelo usuário
    
def alertasVerify(userLat, userLon):
    raioKm = 5


# alertas de simulação:

    alertas = [
        {'lat': userLat + 0.015, 'long': userLon + 0.01, 'tipo': "Enchente"},
        {'lat': userLat + 0.01, 'long': userLon - 0.03, 'tipo': "Ventos fortes"}
    ]

    # A intenção do aplicativo é integrar uma API com detecção de desastres em tempo real e filtrar esses dados para exibição de notificação (como na simulação, com endereço completo)

    mapa = folium.Map(location=[userLat, userLon], zoom_start=13)

    folium.Circle(
        radius=raioKm * 1000,
        location=[userLat, userLon],
        popup="Você está aqui (Raio de 5km)",
        color="blue",
        fill=True,
        fill_opacity=0.1
    ).add_to(mapa)

    folium.Marker(
        [userLat, userLon],
        tooltip="Local do Usuário",
        icon=folium.Icon(color="green")
    ).add_to(mapa)

    # Cria o mapa em html através do banco de dados e informações do Nomatin

    encontrou_alerta = False

    for alerta in alertas:
        distancia = calcular_distancia(userLat, userLon, alerta['lat'], alerta['long'])
        if distancia <= raioKm:
            rua = geolocalizar(alerta['lat'], alerta['long'])
            print(f"⚠️ Alerta de {alerta['tipo']} detectado!\nLocal: {rua}")
            folium.Marker(
                [alerta['lat'], alerta['long']],
                tooltip=f"{alerta['tipo']} ({distancia:.2f} km)",
                popup=rua,
                icon=folium.Icon(color="red", icon="exclamation-sign")
            ).add_to(mapa)
            encontrou_alerta = True

            # Exibe as notificações a cada alerta detectado no raio de 5km da localização inserida pelo usuário e adiciona ao mapa

            if not encontrou_alerta:
                print('Nenhuma anormalidade encontrada! 😊👌')

            mapa.save("alertas_mapa.html")
            print("✅ Mapa salvo como 'alertas_mapa.html'")



try:
    escolha = input("Você deseja usar coordenadas (1) ou nome de rua (2)? ")

    if escolha == "1":
        lat = float(input("Digite sua latitude: "))
        lon = float(input("Digite sua longitude: "))
    elif escolha == "2":
        endereco = input("Digite o nome da rua/endereço: ")
        lat, lon = obter_coordenadas(endereco)
        if lat is None or lon is None:
            raise ValueError("Endereço não encontrado.")
        
        # Caso não haja dados sobre o endereço, exibe o erro

    else:
        raise ValueError("Opção inválida.")
    
    # Caso selecione uma opção inexistente, exibe o erro

    alertasVerify(lat, lon)

except ValueError as e:
    print(f"Erro: {e}")

# Captura erros de digitação do usuário ou a falta de dados sobre o endereço inserido